In [1]:
!pip install datasets transformers jiwer hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00


In [2]:
import os
from datasets import load_dataset
from transformers.models.whisper import WhisperProcessor, WhisperForConditionalGeneration
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from transformers.generation.configuration_utils import GenerationConfig
import torchaudio
import torch
from jiwer import wer, cer

In [3]:
MODEL_PATHS = [
    "victors3136/whisper-ro-multilingual-finetune-it00-es00",
    "victors3136/whisper-ro-multilingual-finetune-it05-es05",
    "victors3136/whisper-ro-multilingual-finetune-it15-es15",
    "victors3136/whisper-ro-multilingual-finetune-it15-es35",
    "victors3136/whisper-ro-multilingual-finetune-it25-es25",
    "victors3136/whisper-ro-multilingual-finetune-it35-es15",
]
LANGUAGE = "ro"
SPLIT = "test"
MAX_SAMPLES = 1_000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
print("Loading Common Voice Romanian test data...")
cv = load_dataset("mozilla-foundation/common_voice_11_0", LANGUAGE, split=SPLIT)
cv = cv.filter(lambda x: x["sentence"] is not None and x["audio"] is not None)
cv = cv.select(range(min(MAX_SAMPLES, len(cv))))

Loading Common Voice Romanian test data...


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 19267it [00:00, 150083.68it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 882it [00:00, 128210.17it/s]


Filter:   0%|          | 0/3859 [00:00<?, ? examples/s]

In [5]:
def speech_file_to_array_fn(batch):
    speech_array, _ = torchaudio.load(batch["audio"]["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["target_text"] = batch["sentence"].lower()
    return batch

In [6]:
cv = cv.map(speech_file_to_array_fn)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
!pip install tqdm

In [11]:
from tqdm import tqdm

In [20]:
results = []
for model_id in MODEL_PATHS:
    fails = 0
    total = 0

    print(f"\nEvaluating {model_id}...")

    processor = AutoProcessor.from_pretrained(model_id)
    model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id) \
                                     .to(DEVICE)

    predictions = []
    references = []

    for sample in tqdm(cv):
        input_features = processor(
            sample["speech"],
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features.to(DEVICE)
        with torch.no_grad():
            try:
                predicted_ids = model.generate(
                    input_features,
                    language="ro",
                    task="transcribe",
                )
            except IndexError as ie:
                fails += 1
                continue
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].lower()

        predictions.append(transcription)
        references.append(sample["target_text"])

    wer_score = wer(references, predictions) * 100
    cer_score = cer(references, predictions) * 100
    print(f"Total samples: {len(cv)}, of which {fails} failed due to indexing errors")
    print(f"{model_id}: WER = {wer_score:.2f}%, CER = {cer_score:.2f}%")
    results.append((model_id, wer_score, cer_score))


results.sort(key=lambda x: (
    x[1], # sort by wer by default
    x[2]) # use cer as tiebreaker
)


Evaluating victors3136/whisper-ro-multilingual-finetune-it00-es00...


100%|██████████| 1000/1000 [05:00<00:00,  3.33it/s]


Total samples: 1000, of which 630 failed due to indexing errors
victors3136/whisper-ro-multilingual-finetune-it00-es00: WER = 350.53%, CER = 204.24%

Evaluating victors3136/whisper-ro-multilingual-finetune-it05-es05...


100%|██████████| 1000/1000 [07:47<00:00,  2.14it/s]


Total samples: 1000, of which 621 failed due to indexing errors
victors3136/whisper-ro-multilingual-finetune-it05-es05: WER = 389.59%, CER = 287.10%

Evaluating victors3136/whisper-ro-multilingual-finetune-it15-es15...


...

Total samples: 1000, of which 44 failed due to indexing errors
victors3136/whisper-ro-multilingual-finetune-it15-es35: WER = 275.69%, CER = 228.74%

Evaluating victors3136/whisper-ro-multilingual-finetune-it25-es25...


100%|██████████| 1000/1000 [04:01<00:00,  4.15it/s]


Total samples: 1000, of which 237 failed due to indexing errors
victors3136/whisper-ro-multilingual-finetune-it25-es25: WER = 113.26%, CER = 111.60%

Evaluating victors3136/whisper-ro-multilingual-finetune-it35-es15...


100%|██████████| 1000/1000 [05:40<00:00,  2.94it/s]

Total samples: 1000, of which 734 failed due to indexing errors
victors3136/whisper-ro-multilingual-finetune-it35-es15: WER = 470.98%, CER = 364.45%


In [19]:
print("\nFinal Benchmark Results:")
print(f"{'Model':<60} {'WER (%)':<10} {'CER (%)':<10}")
for model_id, wer_score, cer_score in results:
    print(f"{model_id:<60} {wer_score:<10.2f} {cer_score:<10.2f}")



Final Benchmark Results:
Model                                                        WER (%)    CER (%)   
victors3136/whisper-ro-multilingual-finetune-it15-es15       104.43     85.22     
victors3136/whisper-ro-multilingual-finetune-it25-es25       113.26     111.60    
victors3136/whisper-ro-multilingual-finetune-it15-es35       275.69     228.74    
victors3136/whisper-ro-multilingual-finetune-it00-es00       350.53     204.24    
victors3136/whisper-ro-multilingual-finetune-it05-es05       389.59     287.10    
victors3136/whisper-ro-multilingual-finetune-it35-es15       470.98     364.45    


In [16]:
len("victors3136/whisper-ro-multilingual-finetune-it15-es15")

54